In [18]:
import os
import pandas as pd
import nltk
import string
import contractions

In [19]:
stop_words = set(nltk.corpus.stopwords.words('english'))
print(stop_words)

{'whom', 'after', 'only', 'no', "won't", 'ours', 'but', 'not', 'to', 'above', 'are', 'by', 'over', 'were', 'through', "mightn't", 'with', 'this', "shouldn't", 'me', 'while', 'again', 'between', 'wasn', 'y', "you'd", 'himself', 'do', 'because', "you're", 'against', 't', 'then', 'him', 'up', 'in', 're', 'being', 'off', "wouldn't", 'mightn', 'out', 'm', 'how', 'you', 'shouldn', 'am', 'his', 'or', 'same', 'so', 'your', 'yourself', 'few', 'has', 'from', 'be', 'before', 'theirs', 'doesn', 'themselves', 'than', 'haven', 'wouldn', 'some', 'didn', 'don', 'each', 'll', "couldn't", 'as', 'there', 'was', 'hasn', 'into', 'what', 'at', "hadn't", 've', 'who', 'did', 'shan', 'during', 'their', 'itself', 'hadn', 'under', 'both', "isn't", 'had', 'weren', "needn't", 'most', 'myself', 'why', 'he', "that'll", 'we', 'that', 'nor', 'couldn', 'd', 'when', "you'll", 'been', 'more', 'own', 'my', 'o', 'can', "don't", 'and', "shan't", 'on', 'a', 'them', 'these', 'once', 'she', 'i', 'now', 'ma', 'hers', 's', 'very

In [20]:
albums = ["Taylor Swift", "Fearless (Taylor's Version)", "Speak Now (Taylor's Version)", 
          "Red (Taylor's Version)", "1989 (Taylor's Version)", "reputation", "Lover", 
          "Midnights", "folklore", "evermore", "The Tortured Poets Department"]

In [21]:
df = pd.read_csv('songs.csv')


In [22]:
in_album = df[df['Album'].isin(albums)]
df = in_album

In [23]:
lemmatizer = nltk.WordNetLemmatizer()

within_description = 0
for lyric in df['Lyrics']:
    word_list=lyric.split()
    word_data = ""
    for word in word_list:
        if "[" or "]" in word:
            if "[" in word and not("]" in word):
                within_description = 1
            elif "]" in word and not("[" in word):
                within_description = 0
            elif "[" in word and "]" in word:
                within_description = 0
            if "(" in word and not(")" in word):
                within_description = 1
            elif ")" in word and not("(" in word):
                within_description = 0
            elif "(" in word and ")" in word:
                within_description = 0
        if not("]" in word or "[" in word or "(" in word or ")" in word) and not(within_description):
            for contraction in contractions.contractions_dict.keys():
                if contraction.lower() in word.lower():
                    word = contractions.fix(word)
            word_data += word + " "

    nltk_tokens = nltk.word_tokenize(text=word_data, language='english', preserve_line=True)

    pronouns_and_not = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 
            'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 
            'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'not', 'no',
            'should', 'be', 'am', 'are', 'is', 'were', 'was', 'being', 'been', 'have', 'whom', 'after', 'only', 'above', 
            'over', 'through', 'with', 'again', 'between', 'because', 'against', 'then', 'off', 'out', 'how', 'same', 'few', 'has', 'from',
            'before', 'than', 'some', 'each', 'there', 'into', 'during', 'under', 'both', 'had', 'most', 'when', 'more', 'own', 'once', 'very',
            'having', 'further', 'other', 'down', 'any', 'should', 'until', 'about', 'will', 'doing', 'do', 'here', 'such', 'too', 'all', 'below',
            'just']
    
    exclude = ['was', 'us']
    
    stop_words = [word for word in stop_words if word not in pronouns_and_not]

    nltk_tokens = [token.lower() for token in nltk_tokens if token.lower() not in stop_words]
    nltk_tokens = [token for token in nltk_tokens if token not in string.punctuation]
    nltk_tokens = [token if token != exclude else lemmatizer.lemmatize(token) for token in nltk_tokens ]
    nltk_tokens = [token for token in nltk_tokens if token != "''" and 
                   token != "``" and token != "'s" and token != "'d" and token != "oh"
                   and token != "hey" and token != "'m" and token != "mm" and token != "oh-oh"
                   and token != "oh-oh-oh" and token != "mmm" and token != "ooh" and token != "ooh-oh"
                   and token != "mm-mm" and token != "la" and token != "ah-ah" and token != "ha"
                   and token != "ha-ah" and token != "di" and token != "da" and token != "da-da" and token != "uh-huh"
                   and token != "ra-di-di-di-di-di-di-di-di-di-da-da" and token != "ah-ah-ah-ah-ah" and token != "ah-ah-ah" and token != "ah-ah-ah-ah"
                   and token != "ah-ah-ah-ah-ah-ah" and token != "ah" and token != "eh" and token != "uh-uh" and token != "ooh-woo-ooh-ooh-ooh" and token != "ooh-ooh-ooh-ooh"
                   and token != "ooh-ooh-ooh" and token != "la-la-la" and token != "la-la-la-la" and token != "woah"]
    nltk_tokens = ["i" if token in {"i-i-i-i", "i-i", "i-i-i"} else token for token in nltk_tokens]
    
    lyric_column = df['Lyrics'] == lyric
    row_index = df.index[lyric_column]
    title = df[df['Lyrics'] == lyric]['Title'].values[0]
    album = df[df['Lyrics'] == lyric]['Album'].values[0]
    file_path = 'songs_excl_ftv/'+ album
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    song = os.path.join(file_path, f"{title}.txt")
    if not("[From the Vault]" in title or "[From The Vault]" in title or "(Remix)" in title):
        with open(song, "w", encoding="utf-8") as f:
            literal = "\n".join(nltk_tokens)
            f.write(literal)
        

